<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/102_TxtSummarizerAgent_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Progress tracking is one of those “agent hygiene” habits** that seems optional at first but becomes critical as agents grow more complex.

---

## 🧭 Why Track Progress?

When you run an agent, it doesn’t just “jump” from goal → result.
It goes through a series of **steps** (plan, read file, generate prompt, summarize, save, etc.).

Without progress tracking:

* You don’t know *where it failed* if something breaks.
* You can’t easily **re-run from the last completed step**.
* You lose the ability to **audit** what the agent did.

With progress tracking:

* You get a running “logbook” of what steps have started, completed, or errored.
* You can debug and improve your agent much faster.
* You have a structured record you can export, display in a UI, or analyze later.

---

## 🔧 The Functions

### 1. `track_progress(ctx, step, status, note="")`

This is the **logger** function.

* It creates or updates a `progress_log` list stored in memory.
* Each entry is a dictionary like:

  ```python
  {"step": "read_txt_file", "status": "completed", "note": "File length: 5530"}
  ```
* The `status` tells you whether the step started, completed, or failed.
* The `note` can carry any extra info (like error messages or metadata).

**Why needed?**
It gives you a **structured trace** of the agent’s journey — like breadcrumbs you can inspect later.

---

### 2. `print_progress(ctx)`

This is a **convenience viewer**.

* It takes the structured log in memory and prints it in a nice format.
* Example output:

  ```
  📊 Progress Log:
  - [started] create_plan (2025-08-26) Planning the steps
  - [completed] create_plan (2025-08-26) 7 steps created
  - [completed] read_txt_file (2025-08-26) File length: 5530
  - [completed] summarize (2025-08-26) Summary saved in memory
  ```

**Why needed?**
Makes debugging and reviewing agent runs much easier (especially in notebooks).

---

## ✅ Summary

* **`track_progress`** = *writes progress updates into memory*
* **`print_progress`** = *reads and displays the log nicely*

They’re not strictly required for a minimal agent, but they make your framework:

* More debuggable
* More user-friendly
* Easier to extend into production agents


In [ ]:
import time

VALID_STATUSES = {"started", "completed", "error"}

def track_progress(ctx, step, status, note=""):
    if status not in VALID_STATUSES:
        raise ValueError(f"Invalid status '{status}'. Use {VALID_STATUSES}.")

    progress = ctx.memory.get("progress_log") or []
    progress.append({
        "step": step,
        "status": status,
        "note": note,
        "time": time.strftime("%Y-%m-%d %H:%M:%S")
    })
    ctx.memory.set("progress_log", progress)

def print_progress(ctx):
    log = ctx.memory.get("progress_log") or []
    print("\n📊 Progress Log:")
    for entry in log:
        print(f"- [{entry['status']}] {entry['step']} ({entry.get('time','')}) {entry['note']}")


## Add to ActionContext?

## option A — keep them as free functions (what you have)

* pros: simple, easy to unit-test in isolation; no class changes
* cons: slightly noisier call sites (`track_progress(ctx, ...)`), functions must know the ctx shape

## option B — make them **methods on `ActionContext`**

* pros: nicer ergonomics, discoverable (`ctx.track_progress(...)`), keeps all “backpack ops” together
* cons: `ActionContext` takes on more responsibility (still fine for a small framework)

given your goal of a reusable *template*, i’d fold them into `ActionContext`. here’s a tidy refactor:


### Why I recommend it (now)

* **Cohesion:** `ctx` is already “the backpack.” Goal, plan, raw\_text, summary all live there; progress about those things belongs there too.
* **Ergonomics:** `ctx.track_progress(...)` is cleaner and discoverable vs free functions that take `ctx`.
* **Encapsulation:** Tools don’t need to know *how* logging works—only that `ctx` can log. If you change the logging strategy, tools don’t change.
* **Testability:** You can unit-test logging by inspecting `ctx.memory["progress_log"]` without extra globals.

### When I’d *not* put it on `ActionContext`

* If you anticipate **multiple sinks** (console + file + DB + telemetry) or structured logging needs, graduate to:

  * `class ProgressLogger` (strategy) and inject as `ctx.logger`
  * `ctx.track_progress(...)` simply delegates to `ctx.logger.log(...)`
* If you want **cross-cutting policies** (PII redaction, correlation IDs, span tracing), a dedicated logger service is better.

### Pragmatic path

1. Put methods on `ActionContext` now (simple, clean).
2. If/when needs grow, refactor `ActionContext.track_progress` to call an injected `logger` without changing any tool code.




In [ ]:
import time

VALID_STATUSES = {"started", "completed", "error"}

class ActionContext:
    def __init__(self, memory, llm, config=None):
        self.memory = memory
        self.llm = llm
        self.config = config or {}

    # --- progress helpers ---
    def track_progress(self, step, status, note=""):
        if status not in VALID_STATUSES:
            raise ValueError(f"Invalid status '{status}'. Use {VALID_STATUSES}.")
        log = self.memory.get("progress_log") or []
        log.append({
            "step": step,
            "status": status,
            "note": note,
            "time": time.strftime("%Y-%m-%d %H:%M:%S"),
        })
        self.memory.set("progress_log", log)

    def print_progress(self):
        log = self.memory.get("progress_log") or []
        print("\n📊 Progress Log:")
        for e in log:
            t = f" ({e.get('time')})" if e.get("time") else ""
            note = f" — {e['note']}" if e.get("note") else ""
            print(f"- [{e['status']}] {e['step']}{t}{note}")

    def last_completed_step(self):
        log = self.memory.get("progress_log") or []
        for e in reversed(log):
            if e.get("status") == "completed":
                return e.get("step")
        return None

    def first_error(self):
        log = self.memory.get("progress_log") or []
        for e in log:
            if e.get("status") == "error":
                return e
        return None




## 🧠 Why `ActionContext`?

Think of `ActionContext` (`ctx`) as the **backpack** your agent carries around.
Inside that backpack:

* **Memory (`ctx.memory`)** → stores all “state” (goal, plan, raw\_text, summary, progress log, etc.)
* **LLM (`ctx.llm`)** → the brain/mind access point
* **Config (`ctx.config`)** → fixed environment details (like input/output folders)
* Potentially more (like dependencies or external APIs later)

---

## 📊 Where Progress Goes

When you call `track_progress(ctx, ...)`, it just writes to:

```python
ctx.memory.set("progress_log", [...])
```

So the **progress log is part of memory**, and memory is carried inside `ctx`.
That means at any point in execution you can inspect progress by peeking into memory:

```python
ctx.memory.get("progress_log")
```

---

## ✅ Why This Matters

1. **Single Source of Truth**
   Everything the agent generates or needs to remember lives inside `ctx.memory`.
   No scattered globals, no hidden state.

2. **Reusability**
   Since tools only ever interact with `ctx`, you can swap in different memory implementations later (e.g. in-memory dict → Redis → database) without rewriting tools.

3. **Inspectability**
   Because progress, goal, plan, and outputs are *all in memory*, you can always log, debug, or export them easily.

---

## 🎯 So yes

You are tracking progress “through” ActionContext, because **ActionContext carries memory, and memory holds the log** (alongside the goal, plan, prompt, etc.).



🔑 — Understanding *state* is a huge step toward “thinking like an agent builder.” Let’s lock it in.

---

## 🧠 Formal Definition of **State**

In computing (and in agents specifically):

> **State** is the collection of all information an agent/system remembers at a given moment, which can affect its future decisions or actions.

* It’s the **snapshot of memory and context** at a specific time.
* If you ran the same code *without state*, it would start “from scratch” each time.
* With state, the agent “knows what happened before” and can adapt accordingly.

---

## 🧩 State in Your Agent

In your design, **state = everything inside `ctx.memory`**:

* `goal` → what the agent is trying to achieve
* `plan` → the step-by-step breakdown
* `raw_text` → the text of the file being summarized
* `summary_prompt` → the LLM-ready summarization prompt
* `summary` → the final LLM output
* `progress_log` → the history of what steps have run and their status

At any point, if you printed `ctx.memory.store`, you’d see the *current state* of the agent.

---

## 🌍 Inside vs Outside State

* **Inside state (memory):** Anything the agent must *remember and use later* in its reasoning or actions (goal, plan, raw text, summary text, progress log).
* **Outside state (environment):** Things the agent *doesn’t carry in memory but interacts with*, like:

  * A summary file written to disk (`summary.txt`)
  * An API call to fetch weather
  * A database update

💡 The agent may *store a reference* to these external things in its state (like “file saved at path: `/content/output/summary.txt`”), but the actual file itself lives in the environment.

---

## ✅ Example to Tie It Together

Imagine you tell the agent: *“Summarize `article.txt` and save the summary.”*

1. **State Before Running**

   ```
   goal = "Summarize article.txt"
   plan = None
   raw_text = None
   summary = None
   progress_log = []
   ```

2. **After Reading File**

   ```
   raw_text = "Article text..."
   progress_log = [{"step":"read_txt_file", "status":"completed"}]
   ```

3. **After Summarizing**

   ```
   summary = "- Bullet points..."
   progress_log = [..., {"step":"summarize", "status":"completed"}]
   ```

4. **After Saving**

   ```
   summary_file = "/content/output/article_summary.txt"
   progress_log = [..., {"step":"save_summary", "status":"completed"}]
   ```

👉 Here the **text** is inside state, but the **file** is outside state (though the path to it might be remembered).

---

## 📌 Short Definition (for your guidebook)

> **State is the total collection of remembered information (in memory) that an agent uses to decide what to do next.**
>
> It includes goals, plans, inputs, outputs, and logs. External actions (like writing to a file) happen outside of state, but references to them can be stored inside state.




In [1]:
!pip -q install openai python-dotenv

from openai import OpenAI
from dotenv import load_dotenv
import os
import textwrap

# ---- Setup ----
load_dotenv('/content/API_KEYS.env')
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise RuntimeError("OPENAI_API_KEY not found in /content/API_KEYS.env")
client = OpenAI(api_key=api_key)

## Replace ActionContext

In [2]:
import os
import textwrap

# ---------------- Memory + Context ----------------
class ScratchMemory:
    def __init__(self):
        self.store = {}

    def get(self, key):
        return self.store.get(key)

    def set(self, key, value):
        self.store[key] = value

# we replace our original ActionContext
# class ActionContext:
#     def __init__(self, memory, llm):
#         self.memory = memory
#         self.llm = llm

import time

VALID_STATUSES = {"started", "completed", "error"}

class ActionContext:
    def __init__(self, memory, llm, config=None):
        self.memory = memory
        self.llm = llm
        self.config = config or {}

    # --- progress helpers ---
    def track_progress(self, step, status, note=""):
        if status not in VALID_STATUSES:
            raise ValueError(f"Invalid status '{status}'. Use {VALID_STATUSES}.")
        log = self.memory.get("progress_log") or []
        log.append({
            "step": step,
            "status": status,
            "note": note,
            "time": time.strftime("%Y-%m-%d %H:%M:%S"),
        })
        self.memory.set("progress_log", log)

    def print_progress(self):
        log = self.memory.get("progress_log") or []
        print("\n📊 Progress Log:")
        for e in log:
            t = f" ({e.get('time')})" if e.get("time") else ""
            note = f" — {e['note']}" if e.get("note") else ""
            print(f"- [{e['status']}] {e['step']}{t}{note}")

    def last_completed_step(self):
        log = self.memory.get("progress_log") or []
        for e in reversed(log):
            if e.get("status") == "completed":
                return e.get("step")
        return None

    def first_error(self):
        log = self.memory.get("progress_log") or []
        for e in log:
            if e.get("status") == "error":
                return e
        return None


In [3]:
# ---------------- LLM Wrapper ----------------
class OpenAILLM:
    def __init__(self, client, model="gpt-4o-mini"):
        self.client = client
        self.model = model

    def complete(self, prompt):
        response = self.client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.3
        )
        return response.choices[0].message.content

# ---------------- Tool: create_plan ----------------
def create_plan(ctx):
    goal = ctx.memory.get("goal")
    if not goal:
        return {"error": "No goal provided in memory."}

    prompt = f"""You are an expert task planner. Given the goal below, break it down into a clear, short list of steps.

Goal: {goal}

Respond with a numbered list of steps."""

    response = ctx.llm.complete(prompt)
    steps = response.strip().split("\n")

    ctx.memory.set("plan", steps)
    return {"message": "Plan created from goal.", "steps": steps}

# ---------------- Tool: read_txt_file ----------------
def read_txt_file(ctx, file_name):
    folder = ctx.config.get("input_folder")
    path = os.path.join(folder, file_name)

    if not os.path.exists(path):
        return {"error": f"File not found: {path}"}

    with open(path, "r", encoding="utf-8") as f:
        text = f.read()

    ctx.memory.set("raw_text", text)
    return {"message": "File read successfully.", "length": len(text)}

In [4]:
# ---------------- Setup + Test ----------------
# Set up memory and config
memory = ScratchMemory()
memory.set("goal", "Summarize the content of a text file.")
config = {
    "input_folder": "/content/files",
    "output_folder": "/content/output"
}

# Set up LLM and context (inject config)
llm = OpenAILLM(client)
ctx = ActionContext(memory=memory, llm=llm, config=config)

os.makedirs(ctx.config["input_folder"], exist_ok=True)
os.makedirs(ctx.config["output_folder"], exist_ok=True)
ctx.track_progress("setup", "completed", "goal + config injected")

In [6]:
# Run planning tool
ctx.track_progress("create_plan", "started", "generating steps")
plan_result = create_plan(ctx)
if "error" in plan_result:
    ctx.track_progress("create_plan", "error", plan_result["error"])
    print("❌ create_plan:", plan_result["error"])
else:
    ctx.track_progress("create_plan", "completed", f"steps={len(plan_result['steps'])}")
    print(plan_result["message"])
    print("\nPlan:")
    for step in plan_result["steps"]:
        wrapped = textwrap.fill(step, width=80, subsequent_indent="  ")
        print(f"- {wrapped}")

# ---------------- Print Goal ----------------
print("\n\n🎯 Goal:")
print(ctx.memory.get("goal"))
print()

# Run file reader tool
file_name = "004_AGENT_Tools.txt"
ctx.track_progress("read_txt_file", "started", f"file={file_name}")
file_result = read_txt_file(ctx, file_name)
if "error" in file_result:
    ctx.track_progress("read_txt_file", "error", file_result["error"])
    print("❌ Error:", file_result["error"])
else:
    ctx.track_progress("read_txt_file", "completed", f"length={file_result['length']}")
    print("\n✅", file_result["message"])
    print(f"Character count: {file_result['length']}\n")

    raw_text = ctx.memory.get("raw_text")
    preview = raw_text[:600]
    wrapped_preview = textwrap.fill(preview, width=80, subsequent_indent="  ")
    print("📄 File Preview:\n")
    print(wrapped_preview)

Plan created from goal.

Plan:
- 1. **Open the Text File**: Use a text editor or programming tool to access the
  file you want to summarize.
- 
- 2. **Read the Content**: Carefully read through the entire text to understand
  the main ideas and themes.
- 
- 3. **Identify Key Points**: Highlight or note down the most important concepts,
  arguments, and facts presented in the text.
- 
- 4. **Determine the Structure**: Organize the key points into a logical
  structure, such as introduction, main ideas, and conclusion.
- 
- 5. **Draft the Summary**: Write a concise summary using your identified key
  points, ensuring to capture the essence of the original text.
- 
- 6. **Review and Edit**: Read through your summary to ensure clarity and
  coherence, making any necessary adjustments for brevity and accuracy.
- 
- 7. **Finalize the Summary**: Save or export the summary in the desired format
  for future reference or sharing.


🎯 Goal:
Summarize the content of a text file.


✅ File read su

In [7]:
# ---------------- Tool: generate_summary_prompt ----------------
def generate_summary_prompt(ctx):
    text = ctx.memory.get("raw_text")
    if not text:
        return {"error": "No raw text found in memory."}

    # Optional: truncate text if it's too long
    max_len = 2000
    short_text = text[:max_len]

    prompt = f"""You are an expert technical writer.

Summarize the following content into a set of clear, concise bullet points. Focus on the main ideas, and skip boilerplate or excessive detail.

Text:
\"\"\"
{short_text}
\"\"\"

Summary:"""

    ctx.memory.set("summary_prompt", prompt)
    return {"message": "Summary prompt created.", "prompt_preview": prompt[:600]}

# ---------------- Run Prompt Generator ----------------
ctx.track_progress("generate_summary_prompt", "started")
prompt_result = generate_summary_prompt(ctx)
if "error" in prompt_result:
    ctx.track_progress("generate_summary_prompt", "error", prompt_result["error"])
    print("❌", prompt_result["error"])
else:
    ctx.track_progress("generate_summary_prompt", "completed")
    print("🛠️", prompt_result["message"])
    print("\n🧾 Prompt Preview:\n")
    print(textwrap.fill(prompt_result["prompt_preview"], width=80, subsequent_indent="  "))

# ---------------- Tool: summarize ----------------
def summarize(ctx):
    prompt = ctx.memory.get("summary_prompt")
    if not prompt:
        return {"error": "No summary prompt found in memory."}
    try:
        response = ctx.llm.complete(prompt)
    except Exception as e:
        return {"error": f"LLM error: {e}"}

    ctx.memory.set("summary", response)
    return {"message": "Summary completed.", "summary_preview": response[:1000]}

# ---------------- Run summarization ----------------
ctx.track_progress("summarize", "started")
summary_result = summarize(ctx)

print("\n🧠 LLM Output:")
if "error" in summary_result:
    ctx.track_progress("summarize", "error", summary_result["error"])
    print("❌ Error:", summary_result["error"])
else:
    ctx.track_progress("summarize", "completed")
    print("✅", summary_result["message"])
    print("\n📝 Summary Preview:\n")
    print(textwrap.fill(summary_result["summary_preview"], width=80, subsequent_indent="  "))

# ---------------- Print ActionContext Overview ----------------
print("\n" + "="*80)
print("📦 ActionContext Snapshot")

# Memory contents
print("\n🧠 Memory:")
for key, value in ctx.memory.store.items():
    display = str(value)
    if isinstance(value, str) and len(display) > 400:
        display = display[:400] + "..."
    print(f"  {key}: {display}")

# Config
print("\n⚙️ Config:")
print(f"  {ctx.config}")

# LLM Info
print("\n🧩 LLM:")
if ctx.llm:
    print(f"  Type: {ctx.llm.__class__.__name__}")
    print(f"  Model: {ctx.llm.model}")
else:
    print("  No LLM connected.")

# Progress log (optional)
ctx.print_progress()


🛠️ Summary prompt created.

🧾 Prompt Preview:

You are an expert technical writer.  Summarize the following content into a set
  of clear, concise bullet points. Focus on the main ideas, and skip boilerplate
  or excessive detail.  Text: """  #===========AI-Agent Tool Descriptions and
  Naming  Describing Tools to the Agent  When developing an agentic AI system,
  one of the most critical aspects is ensuring that the agent understands the
  tools it has access to. In our previous tutorial, we explored how an AI agent
  interacts with an environment. Now, we extend that discussion to focus on tool
  definition, particularly the importance of naming, para

🧠 LLM Output:
✅ Summary completed.

📝 Summary Preview:

- **Importance of Tool Understanding**: AI agents must clearly understand the
  tools available to them for effective operation. - **Example Use Case**: An AI
  agent automates documentation generation for Python code by scanning files in
  a `src/` directory and creating document

This looks great — you’ve got a clean end-to-end run:

* ✅ Goal, plan, raw\_text, summary\_prompt, and summary are all present in memory.
* ✅ Progress log tells a clear story (first file missing → retried with another → success).
* ✅ Prompt → summary flow is working as intended.

A few small, high-leverage tweaks:

1. Plan parsing (you’ve got blank lines)

* Your plan array includes empty strings because we split on `\n`.
* Quick fix: normalize to a clean list of steps.

```python
import re
raw_steps = plan_result["steps"]
clean_steps = [s.strip("-• ").strip() for s in raw_steps if s.strip()]
# or, if you store raw text: steps = re.findall(r"\d+\.\s+(.*)", response, re.M)
ctx.memory.set("plan", clean_steps)
```

2. Keep the file name in memory

* Helps with saving and provenance:

```python
ctx.memory.set("file_name", file_name)
```

3. Flag when you truncate the source

* You cap at 2,000 chars; record that for transparency.

```python
max_len = 2000
text = ctx.memory.get("raw_text")
truncated = len(text) > max_len
short_text = text[:max_len]
ctx.memory.set("was_truncated", truncated)
```

4. (Optional) Slightly lower temperature for consistency

* `temperature=0.2` often yields tighter bullets for summarization.

5. Next step: save the summary to disk (close the loop)
   Here’s a tiny, robust `save_summary` that uses your injected `output_folder` and the input `file_name`:

```python
import os

def save_summary(ctx, out_name=None):
    ctx.track_progress("save_summary", "started")
    summary = ctx.memory.get("summary")
    if not summary:
        err = "No summary in memory."
        ctx.track_progress("save_summary", "error", err)
        return {"error": err}

    output_folder = ctx.config.get("output_folder")
    if not output_folder:
        err = "No output_folder in config."
        ctx.track_progress("save_summary", "error", err)
        return {"error": err}

    os.makedirs(output_folder, exist_ok=True)

    base = out_name
    if not base:
        src = ctx.memory.get("file_name", "summary.txt")
        root, _ = os.path.splitext(os.path.basename(src))
        base = f"{root}_summary.txt"

    path = os.path.join(output_folder, base)
    with open(path, "w", encoding="utf-8") as f:
        f.write(summary)

    ctx.track_progress("save_summary", "completed", f"path={path}")
    ctx.memory.set("summary_path", path)
    return {"message": "Summary saved.", "path": path}
```

Usage right after `summarize(ctx)`:

```python
res = save_summary(ctx)
print(res.get("message", res.get("error")))
```

6. Tiny inspection helpers you can use anytime

```python
print("Last completed:", ctx.last_completed_step())
ctx.print_progress()
```

If you want, we can drop `save_summary` into your notebook now and do one final run so your agent goes goal → plan → read → prompt → summarize → **save** → log.


# Final Code (Working Copy)


In [8]:
!pip -q install openai python-dotenv

from openai import OpenAI
from dotenv import load_dotenv
import os
import textwrap
import time
import re

# ---- Setup ----
load_dotenv('/content/API_KEYS.env')
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise RuntimeError("OPENAI_API_KEY not found in /content/API_KEYS.env")
client = OpenAI(api_key=api_key)


# ---------------- Memory + Context ----------------
class ScratchMemory:
    def __init__(self):
        self.store = {}

    def get(self, key):
        return self.store.get(key)

    def set(self, key, value):
        self.store[key] = value

VALID_STATUSES = {"started", "completed", "error"}

class ActionContext:
    def __init__(self, memory, llm, config=None):
        self.memory = memory
        self.llm = llm
        self.config = config or {}

    # --- progress helpers ---
    def track_progress(self, step, status, note=""):
        if status not in VALID_STATUSES:
            raise ValueError(f"Invalid status '{status}'. Use {VALID_STATUSES}.")
        log = self.memory.get("progress_log") or []
        log.append({
            "step": step,
            "status": status,
            "note": note,
            "time": time.strftime("%Y-%m-%d %H:%M:%S"),
        })
        self.memory.set("progress_log", log)

    def print_progress(self):
        log = self.memory.get("progress_log") or []
        print("\n📊 Progress Log:")
        for e in log:
            t = f" ({e.get('time')})" if e.get("time") else ""
            note = f" — {e['note']}" if e.get("note") else ""
            print(f"- [{e['status']}] {e['step']}{t}{note}")

    def last_completed_step(self):
        log = self.memory.get("progress_log") or []
        for e in reversed(log):
            if e.get("status") == "completed":
                return e.get("step")
        return None

    def first_error(self):
        log = self.memory.get("progress_log") or []
        for e in log:
            if e.get("status") == "error":
                return e
        return None

# ---------------- LLM Wrapper ----------------
class OpenAILLM:
    def __init__(self, client, model="gpt-4o-mini"):
        self.client = client
        self.model = model

    def complete(self, prompt):
        response = self.client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.3
        )
        return response.choices[0].message.content

# ---------------- Tool: create_plan ----------------
# def create_plan(ctx):
#     goal = ctx.memory.get("goal")
#     if not goal:
#         return {"error": "No goal provided in memory."}

#     prompt = f"""You are an expert task planner. Given the goal below, break it down into a clear, short list of steps.

# Goal: {goal}

# Respond with a numbered list of steps."""

#     response = ctx.llm.complete(prompt)
#     steps = response.strip().split("\n")

#     ctx.memory.set("plan", steps)
#     return {"message": "Plan created from goal.", "steps": steps}


def create_plan(ctx):
    goal = ctx.memory.get("goal")
    if not goal:
        return {"error": "No goal provided in memory."}

    prompt = f"""You are an expert task planner. Given the goal below, break it down into a clear, short list of steps.

Goal: {goal}

Respond with a numbered list of steps."""
    raw = ctx.llm.complete(prompt).strip()

    # Prefer numbered steps like "1. ...", "2) ..."
    numbered = re.findall(r'^\s*(?:\d+[\).\s-]+)\s*(.+)$', raw, flags=re.M)

    if numbered:
        steps = numbered
    else:
        # Fallback to bullets like "- ...", "* ...", "• ..."
        bullets = re.findall(r'^\s*(?:[-*•]\s+)(.+)$', raw, flags=re.M)
        steps = bullets if bullets else [ln.strip() for ln in raw.splitlines() if ln.strip()]

    # Normalize: collapse spaces, trim punctuation, drop empties/dupes
    norm = []
    seen = set()
    for s in steps:
        s = re.sub(r'\s+', ' ', s).strip(' .')
        if s and s.lower() not in seen:
            seen.add(s.lower())
            norm.append(s)

    ctx.memory.set("plan", norm)
    return {"message": "Plan created from goal.", "steps": norm}

'''
What changed

We parse the model’s text into clean steps (regex for numbered/bulleted lines).
We normalize: trim, collapse whitespace, drop blanks/dupes.

What didn’t change

The prompt text and the LLM call are the same.
The model’s reasoning/content isn’t altered—just formatted for reliable downstream use.
'''

# ---------------- Tool: read_txt_file ----------------
# def read_txt_file(ctx, file_name):
#     folder = ctx.config.get("input_folder")
#     path = os.path.join(folder, file_name)

#     if not os.path.exists(path):
#         return {"error": f"File not found: {path}"}

#     with open(path, "r", encoding="utf-8") as f:
#         text = f.read()

#     ctx.memory.set("raw_text", text)
#     return {"message": "File read successfully.", "length": len(text)}

def read_txt_file(ctx, file_name):
    ctx.track_progress("read_txt_file", "started", f"file={file_name}")

    folder = ctx.config.get("input_folder")
    path = os.path.join(folder, file_name)

    if not os.path.exists(path):
        ctx.track_progress("read_txt_file", "error", f"missing={path}")
        return {"error": f"File not found: {path}"}

    with open(path, "r", encoding="utf-8") as f:
        text = f.read()

    # Record state
    ctx.memory.set("file_name", file_name)
    ctx.memory.set("raw_text", text)

    ctx.track_progress("read_txt_file", "completed", f"length={len(text)}")
    return {"message": "File read successfully.", "length": len(text)}

''' file_name is run-time state (it can change per task), not static environment.
So it belongs in memory (ctx.memory), not in config (which is for constants like folders).'''

# ---------------- Setup + Test ----------------
# Set up memory and config
memory = ScratchMemory()
memory.set("goal", "Summarize the content of a text file.")
config = {
    "input_folder": "/content/files",
    "output_folder": "/content/output"
}

# Set up LLM and context (inject config)
llm = OpenAILLM(client)
ctx = ActionContext(memory=memory, llm=llm, config=config)

os.makedirs(ctx.config["input_folder"], exist_ok=True)
os.makedirs(ctx.config["output_folder"], exist_ok=True)
ctx.track_progress("setup", "completed", "goal + config injected")

# Run planning tool
ctx.track_progress("create_plan", "started", "generating steps")
plan_result = create_plan(ctx)
if "error" in plan_result:
    ctx.track_progress("create_plan", "error", plan_result["error"])
    print("❌ create_plan:", plan_result["error"])
else:
    ctx.track_progress("create_plan", "completed", f"steps={len(plan_result['steps'])}")
    print(plan_result["message"])
    print("\nPlan:")
    for step in plan_result["steps"]:
        wrapped = textwrap.fill(step, width=80, subsequent_indent="  ")
        print(f"- {wrapped}")

# ---------------- Print Goal ----------------
print("\n\n🎯 Goal:")
print(ctx.memory.get("goal"))
print()

# Run file reader tool
file_name = "004_AGENT_Tools.txt"
ctx.track_progress("read_txt_file", "started", f"file={file_name}")
file_result = read_txt_file(ctx, file_name)
if "error" in file_result:
    ctx.track_progress("read_txt_file", "error", file_result["error"])
    print("❌ Error:", file_result["error"])
else:
    ctx.track_progress("read_txt_file", "completed", f"length={file_result['length']}")
    print("\n✅", file_result["message"])
    print(f"Character count: {file_result['length']}\n")

    raw_text = ctx.memory.get("raw_text")
    preview = raw_text[:600]
    wrapped_preview = textwrap.fill(preview, width=80, subsequent_indent="  ")
    print("📄 File Preview:\n")
    print(wrapped_preview)

# ---------------- Tool: generate_summary_prompt ----------------
# def generate_summary_prompt(ctx):
#     text = ctx.memory.get("raw_text")
#     if not text:
#         return {"error": "No raw text found in memory."}

#     # Optional: truncate text if it's too long
#     max_len = 2000
#     short_text = text[:max_len]

#     prompt = f"""You are an expert technical writer.

# Summarize the following content into a set of clear, concise bullet points. Focus on the main ideas, and skip boilerplate or excessive detail.

# Text:
# \"\"\"
# {short_text}
# \"\"\"

# Summary:"""

#     ctx.memory.set("summary_prompt", prompt)
#     return {"message": "Summary prompt created.", "prompt_preview": prompt[:600]}

def generate_summary_prompt(ctx, max_len=2000):
    text = ctx.memory.get("raw_text")
    if not text:
        return {"error": "No raw text found in memory."}

    # Truncate + record bookkeeping
    truncated = len(text) > max_len
    short_text = text[:max_len]
    ctx.memory.set("was_truncated", truncated)
    ctx.memory.set("source_length", len(text))
    ctx.memory.set("used_length", len(short_text))

    prompt = f"""You are an expert technical writer.

Summarize the following content into a set of clear, concise bullet points. Focus on the main ideas, and skip boilerplate or excessive detail.

Text:
\"\"\"
{short_text}
\"\"\"

Summary:"""

    ctx.memory.set("summary_prompt", prompt)
    return {
        "message": "Summary prompt created.",
        "prompt_preview": prompt[:600],
        "truncated": truncated,
        "used": len(short_text),
        "total": len(text),
    }


# ---------------- Run Prompt Generator ----------------
ctx.track_progress("generate_summary_prompt", "started")
prompt_result = generate_summary_prompt(ctx)
if "error" in prompt_result:
    ctx.track_progress("generate_summary_prompt", "error", prompt_result["error"])
    print("❌", prompt_result["error"])
else:
    ctx.track_progress("generate_summary_prompt", "completed")
    print("🛠️", prompt_result["message"])
    print("\n🧾 Prompt Preview:\n")
    print(textwrap.fill(prompt_result["prompt_preview"], width=80, subsequent_indent="  "))

# ---------------- Tool: summarize ----------------
def summarize(ctx):
    prompt = ctx.memory.get("summary_prompt")
    if not prompt:
        return {"error": "No summary prompt found in memory."}
    try:
        response = ctx.llm.complete(prompt)
    except Exception as e:
        return {"error": f"LLM error: {e}"}

    ctx.memory.set("summary", response)
    return {"message": "Summary completed.", "summary_preview": response[:1000]}

# ---------------- Run summarization ----------------
ctx.track_progress("summarize", "started")
summary_result = summarize(ctx)

print("\n🧠 LLM Output:")
if "error" in summary_result:
    ctx.track_progress("summarize", "error", summary_result["error"])
    print("❌ Error:", summary_result["error"])
else:
    ctx.track_progress("summarize", "completed")
    print("✅", summary_result["message"])
    print("\n📝 Summary Preview:\n")
    print(textwrap.fill(summary_result["summary_preview"], width=80, subsequent_indent="  "))

# ---------------- Print ActionContext Overview ----------------
print("\n" + "="*80)
print("📦 ActionContext Snapshot")

# Memory contents
print("\n🧠 Memory:")
for key, value in ctx.memory.store.items():
    display = str(value)
    if isinstance(value, str) and len(display) > 400:
        display = display[:400] + "..."
    print(f"  {key}: {display}")

# Config
print("\n⚙️ Config:")
print(f"  {ctx.config}")

# LLM Info
print("\n🧩 LLM:")
if ctx.llm:
    print(f"  Type: {ctx.llm.__class__.__name__}")
    print(f"  Model: {ctx.llm.model}")
else:
    print("  No LLM connected.")

# Progress log (optional)
ctx.print_progress()


Plan created from goal.

Plan:
- **Open the Text File**: Use a text editor or programming tool to access the file
  containing the content you want to summarize
- **Read the Content**: Carefully read through the entire text to understand the
  main ideas and themes
- **Identify Key Points**: Highlight or note down the main arguments, concepts,
  and any important details that support them
- **Organize Information**: Group similar ideas together and determine the logical
  flow of the content
- **Draft the Summary**: Write a concise summary that captures the essence of the
  text, focusing on the key points identified
- **Review and Edit**: Reread the summary to ensure clarity and coherence, making
  any necessary adjustments for conciseness and accuracy
- **Finalize the Summary**: Save or present the summary in the desired format,
  ensuring it is easy to understand and free of errors


🎯 Goal:
Summarize the content of a text file.


✅ File read successfully.
Character count: 3107

📄 F

#FINAL CODE

In [10]:
!pip -q install openai python-dotenv

from openai import OpenAI
from dotenv import load_dotenv
import os
import textwrap
import time
import re

# ---- Setup ----
load_dotenv('/content/API_KEYS.env')
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise RuntimeError("OPENAI_API_KEY not found in /content/API_KEYS.env")
client = OpenAI(api_key=api_key)


# ---------------- Memory + Context ----------------
class ScratchMemory:
    def __init__(self):
        self.store = {}

    def get(self, key):
        return self.store.get(key)

    def set(self, key, value):
        self.store[key] = value

VALID_STATUSES = {"started", "completed", "error"}

class ActionContext:
    def __init__(self, memory, llm, config=None):
        self.memory = memory
        self.llm = llm
        self.config = config or {}

    # --- progress helpers ---
    def track_progress(self, step, status, note=""):
        if status not in VALID_STATUSES:
            raise ValueError(f"Invalid status '{status}'. Use {VALID_STATUSES}.")
        log = self.memory.get("progress_log") or []
        log.append({
            "step": step,
            "status": status,
            "note": note,
            "time": time.strftime("%Y-%m-%d %H:%M:%S"),
        })
        self.memory.set("progress_log", log)

    def print_progress(self):
        log = self.memory.get("progress_log") or []
        print("\n📊 Progress Log:")
        for e in log:
            t = f" ({e.get('time')})" if e.get("time") else ""
            note = f" — {e['note']}" if e.get("note") else ""
            print(f"- [{e['status']}] {e['step']}{t}{note}")

    def last_completed_step(self):
        log = self.memory.get("progress_log") or []
        for e in reversed(log):
            if e.get("status") == "completed":
                return e.get("step")
        return None

    def first_error(self):
        log = self.memory.get("progress_log") or []
        for e in log:
            if e.get("status") == "error":
                return e
        return None

# ---------------- LLM Wrapper ----------------
class OpenAILLM:
    def __init__(self, client, model="gpt-4o-mini", temperature=0.2):
        self.client = client
        self.model = model
        self.temperature = temperature

    def complete(self, prompt, **kwargs):
        temp = kwargs.get("temperature", self.temperature)
        resp = self.client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}],
            temperature=temp,
        )
        return resp.choices[0].message.content

# ---------------- Tool: create_plan ----------------
def create_plan(ctx):
    goal = ctx.memory.get("goal")
    if not goal:
        return {"error": "No goal provided in memory."}

    prompt = f"""You are an expert task planner. Given the goal below, break it down into a clear, short list of steps.

Goal: {goal}

Respond ONLY with a numbered list, one step per line. No extra prose."""
    raw = ctx.llm.complete(prompt).strip()

    # Prefer numbered steps like "1. ...", "2) ..."
    numbered = re.findall(r'^\s*(?:\d+[\).\s-]+)\s*(.+)$', raw, flags=re.M)

    if numbered:
        steps = numbered
    else:
        # Fallback to bullets like "- ...", "* ...", "• ..."
        bullets = re.findall(r'^\s*(?:[-*•]\s+)(.+)$', raw, flags=re.M)
        steps = bullets if bullets else [ln.strip() for ln in raw.splitlines() if ln.strip()]

    # Normalize: collapse spaces, trim punctuation, drop empties/dupes
    norm = []
    seen = set()
    for s in steps:
        s = re.sub(r'\s+', ' ', s).strip(' .')
        if s and s.lower() not in seen:
            seen.add(s.lower())
            norm.append(s)

    ctx.memory.set("plan", norm)
    return {"message": "Plan created from goal.", "steps": norm}

# ---------------- Tool: read_txt_file ----------------
def read_txt_file(ctx, file_name):
    ctx.track_progress("read_txt_file", "started", f"file={file_name}")

    folder = ctx.config.get("input_folder")
    path = os.path.join(folder, file_name)

    if not os.path.exists(path):
        msg = f"File not found: {path}"
        ctx.track_progress("read_txt_file", "error", msg)
        return {"error": msg}

    with open(path, "r", encoding="utf-8") as f:
        text = f.read()

    ctx.memory.set("file_name", file_name)
    ctx.memory.set("raw_text", text)

    ctx.track_progress("read_txt_file", "completed", f"length={len(text)}")
    return {"message": "File read successfully.", "length": len(text)}

# ---------------- Setup + Test ----------------
# Set up memory and config
memory = ScratchMemory()
memory.set("goal", "Summarize the content of a text file.")
config = {
    "input_folder": "/content/files",
    "output_folder": "/content/output"
}

# Set up LLM and context (inject config)
# Setup
llm = OpenAILLM(
    client,
    model=config.get("model", "gpt-4o-mini"),
    temperature=config.get("temperature", 0.2),
)

# Set up ActionContext
ctx = ActionContext(memory=memory, llm=llm, config=config)

os.makedirs(ctx.config["input_folder"], exist_ok=True)
os.makedirs(ctx.config["output_folder"], exist_ok=True)
ctx.track_progress("setup", "completed", "goal + config injected")

# Run planning tool
ctx.track_progress("create_plan", "started", "generating steps")
plan_result = create_plan(ctx)
if "error" in plan_result:
    ctx.track_progress("create_plan", "error", plan_result["error"])
    print("❌ create_plan:", plan_result["error"])
else:
    ctx.track_progress("create_plan", "completed", f"steps={len(plan_result['steps'])}")
    print(plan_result["message"])
    print("\nPlan:")
    for step in plan_result["steps"]:
        wrapped = textwrap.fill(step, width=80, subsequent_indent="  ")
        print(f"- {wrapped}")

# ---------------- Print Goal ----------------
print("\n\n🎯 Goal:")
print(ctx.memory.get("goal"))
print()

# Run file reader tool
file_name = "004_AGENT_Tools.txt"

file_result = read_txt_file(ctx, file_name)

if "error" in file_result:
    print("❌ Error:", file_result["error"])
else:
    print("\n✅", file_result["message"])
    print(f"Character count: {file_result['length']}\n")

    raw_text = ctx.memory.get("raw_text")
    preview = raw_text[:600]
    print("📄 File Preview:\n")
    print(textwrap.fill(preview, width=80, subsequent_indent="  "))


# ---------------- Tool: generate_summary_prompt ----------------
def generate_summary_prompt(ctx, max_len=2000):
    text = ctx.memory.get("raw_text")
    if not text:
        return {"error": "No raw text found in memory."}

    # Truncate + record bookkeeping
    truncated = len(text) > max_len
    short_text = text[:max_len]
    ctx.memory.set("was_truncated", truncated)
    ctx.memory.set("source_length", len(text))
    ctx.memory.set("used_length", len(short_text))

    prompt = f"""You are an expert technical writer.

Summarize the following content into a set of clear, concise bullet points. Focus on the main ideas, and skip boilerplate or excessive detail.

Text:
\"\"\"
{short_text}
\"\"\"

Summary:"""

    ctx.memory.set("summary_prompt", prompt)
    return {
        "message": "Summary prompt created.",
        "prompt_preview": prompt[:600],
        "truncated": truncated,
        "used": len(short_text),
        "total": len(text),
    }

# ---------------- Tool: save_summary ----------------
def save_summary(ctx, out_name=None):
    ctx.track_progress("save_summary", "started")
    summary = ctx.memory.get("summary")
    if not summary:
        err = "No summary in memory."
        ctx.track_progress("save_summary", "error", err)
        return {"error": err}

    out_dir = ctx.config.get("output_folder")
    if not out_dir:
        err = "No output_folder in config."
        ctx.track_progress("save_summary", "error", err)
        return {"error": err}

    os.makedirs(out_dir, exist_ok=True)
    base = out_name
    if not base:
        src = ctx.memory.get("file_name", "summary.txt")
        root, _ = os.path.splitext(os.path.basename(src))
        base = f"{root}_summary.txt"
    path = os.path.join(out_dir, base)

    with open(path, "w", encoding="utf-8") as f:
        f.write(summary)

    ctx.memory.set("summary_path", path)
    ctx.track_progress("save_summary", "completed", f"path={path}")
    return {"message": "Summary saved.", "path": path}

# ---------------- Run Prompt Generator ----------------
ctx.track_progress("generate_summary_prompt", "started")
prompt_result = generate_summary_prompt(ctx)
if "error" in prompt_result:
    ctx.track_progress("generate_summary_prompt", "error", prompt_result["error"])
    print("❌", prompt_result["error"])
else:
    ctx.track_progress("generate_summary_prompt", "completed")
    print("🛠️", prompt_result["message"])
    if prompt_result.get("truncated"):
        print(f"(note) Prompt truncated to {prompt_result['used']} / {prompt_result['total']} chars.")
    print("\n🧾 Prompt Preview:\n")
    print(textwrap.fill(prompt_result["prompt_preview"], width=80, subsequent_indent="  "))

# ---------------- Tool: summarize ----------------
def summarize(ctx):
    prompt = ctx.memory.get("summary_prompt")
    if not prompt:
        return {"error": "No summary prompt found in memory."}
    try:
        response = ctx.llm.complete(prompt)
    except Exception as e:
        return {"error": f"LLM error: {e}"}

    ctx.memory.set("summary", response)
    return {"message": "Summary completed.", "summary_preview": response[:1000]}

res = save_summary(ctx)
print(res.get("message", res.get("error")))

# ---------------- Run summarization ----------------
ctx.track_progress("summarize", "started")
summary_result = summarize(ctx)
if "error" in summary_result:
    ctx.track_progress("summarize", "error", summary_result["error"])
    print("❌ Error:", summary_result["error"])
else:
    ctx.track_progress("summarize", "completed")
    print("\n✅", summary_result["message"])
    print("\n📝 Summary Preview:\n")
    print(textwrap.fill(summary_result["summary_preview"], width=80, subsequent_indent="  "))


# ---------------- Print ActionContext Overview ----------------
print("\n" + "="*80)
print("📦 ActionContext Snapshot")

# Memory contents
print("\n🧠 Memory:")
for key, value in ctx.memory.store.items():
    display = str(value)
    if isinstance(value, str) and len(display) > 400:
        display = display[:400] + "..."
    print(f"  {key}: {display}")

# Config
print("\n⚙️ Config:")
print(f"  {ctx.config}")

# LLM Info
print("\n🧩 LLM:")
if ctx.llm:
    print(f"  Type: {ctx.llm.__class__.__name__}")
    print(f"  Model: {ctx.llm.model}")
else:
    print("  No LLM connected.")

# Progress log (optional)
ctx.print_progress()


Plan created from goal.

Plan:
- Open the text file using a text editor or programming tool
- Read the content of the text file
- Identify the main ideas and key points in the text
- Take notes on important details and supporting information
- Organize the notes into a coherent structure
- Write a concise summary based on the organized notes
- Review and edit the summary for clarity and accuracy


🎯 Goal:
Summarize the content of a text file.


✅ File read successfully.
Character count: 3107

📄 File Preview:

 #===========AI-Agent Tool Descriptions and Naming  Describing Tools to the
  Agent  When developing an agentic AI system, one of the most critical aspects
  is ensuring that the agent understands the tools it has access to. In our
  previous tutorial, we explored how an AI agent interacts with an environment.
  Now, we extend that discussion to focus on tool definition, particularly the
  importance of naming, parameters, and structured metadata.  Example:
  Automating Documentat